# Amazon Bedrock AgentCore 런타임(Runtime)에서 MCP 서버 호스팅 - OAuth 인바운드 인증(Authentication)

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)에서 MCP(Model Context Protocol) 서버를 호스팅하는 방법을 배웁니다. Amazon Bedrock AgentCore Python SDK를 사용하여 MCP 도구(Tool)를 Amazon Bedrock AgentCore와 호환되는 MCP 서버로 래핑합니다.

Amazon Bedrock AgentCore Python SDK는 MCP 서버 구현 세부사항을 처리하므로 도구(Tool)의 핵심 기능에 집중할 수 있습니다. 코드를 직접 통신을 위한 AgentCore 표준화된 MCP 프로토콜 계약으로 변환합니다.

### 튜토리얼 세부사항

| 정보                | 세부사항                                                   |
|:--------------------|:----------------------------------------------------------|
| 튜토리얼 유형       | 도구(Tool) 호스팅                                          |
| 도구(Tool) 유형     | MCP 서버                                                   |
| 튜토리얼 구성 요소   | AgentCore 런타임(Runtime)에 MCP 서버 호스팅                |
| 튜토리얼 분야       | 범용                                                       |
| 예제 복잡도         | 쉬움                                                       |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 MCP                  |

### 튜토리얼 아키텍처

이 튜토리얼에서는 MCP 서버를 AgentCore 런타임(Runtime)에 배포(Deployment)하는 방법을 설명합니다.

데모 목적으로 `add_numbers`, `multiply_numbers`, `greet_user` 3가지 도구(Tool)가 있는 간단한 MCP 서버를 사용합니다.

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* 커스텀 도구(Tool)로 MCP 서버 생성
* 로컬에서 MCP 서버 테스트
* Amazon Bedrock AgentCore 런타임(Runtime)에 MCP 서버 호스팅
* 인증(Authentication)으로 배포(Deployment)된 MCP 서버 호출(Invocation)

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명 구성(Configuration)
* Amazon Bedrock AgentCore SDK
* MCP(Model Context Protocol) 라이브러리
* Docker 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os
import sys

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool

In [ ]:
boto_session = Session()
region = boto_session.region_name

ssm_client = boto_session.client('ssm', region_name=region)
secrets_client = boto_session.client('secretsmanager', region_name=region)
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)

tool_name = "mcp_server_agentcore"

## MCP(Model Context Protocol) 이해하기

MCP는 AI 모델이 외부 데이터와 도구(Tool)에 안전하게 액세스할 수 있게 해주는 프로토콜입니다. 주요 개념:

* **도구(Tool)**: AI가 작업을 수행하기 위해 호출할 수 있는 함수
* **스트리밍 가능한 HTTP**: AgentCore 런타임(Runtime)에서 사용하는 전송 프로토콜
* **세션 격리**: 각 클라이언트는 `Mcp-Session-Id` 헤더를 통해 격리된 세션을 받음
* **무상태 운영**: 서버는 확장성을 위해 무상태 운영을 지원해야 함

AgentCore 런타임(Runtime)은 MCP 서버가 기본 경로로 `0.0.0.0:8000/mcp`에서 호스팅될 것으로 예상합니다.

### 프로젝트 구조

적절한 구조로 프로젝트를 설정해 보겠습니다:

```
mcp_server_project/
├── mcp_server.py              # 메인 MCP 서버 코드
├── my_mcp_client.py          # 로컬 테스트 클라이언트
├── my_mcp_client_remote.py   # 원격 테스트 클라이언트
├── requirements.txt          # 종속성
└── __init__.py              # Python 패키지 마커
```

## MCP 서버 생성

이제 세 가지 간단한 도구(Tool)로 MCP 서버를 생성해 보겠습니다. 서버는 AgentCore 런타임(Runtime) 호환성을 위해 필요한 `stateless_http=True`로 FastMCP를 사용합니다.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### 이 코드가 하는 일

* **FastMCP**: 도구(Tool)를 호스팅할 수 있는 MCP 서버 생성
* **@mcp.tool()**: Python 함수를 MCP 도구(Tool)로 변환하는 데코레이터
* **stateless_http=True**: AgentCore 런타임(Runtime) 호환성에 필요
* **도구(Tool)**: 다양한 유형의 작업을 보여주는 세 가지 간단한 도구(Tool)

## 로컬 테스트 클라이언트 생성

AgentCore 런타임(Runtime)에 배포(Deployment)하기 전에 로컬에서 MCP 서버를 테스트할 클라이언트를 생성해 보겠습니다:

In [ ]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### 로컬에서 테스트

MCP 서버를 로컬에서 테스트하려면:

1. **터미널 1**: MCP 서버 시작
   ```bash
   python mcp_server.py
   ```
   
2. **터미널 2**: 테스트 클라이언트 실행
   ```bash
   python my_mcp_client.py
   ```

출력에서 세 가지 도구(Tool)가 나열되어 있는 것을 볼 수 있습니다.

## 인증(Authentication)을 위한 Amazon Cognito 설정

AgentCore 런타임(Runtime)은 인증(Authentication)이 필요합니다. Amazon Cognito를 사용하여 배포(Deployment)된 MCP 서버에 액세스하기 위한 JWT 토큰을 제공합니다.

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

## AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configuration)

다음으로 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configuration)합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 키트를 구성(Configuration)합니다.

구성(Configuration) 단계에서 애플리케이션 코드를 기반으로 도커 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

## AgentCore 런타임(Runtime)에 MCP 서버 시작

이제 도커 파일이 준비되었으니 MCP 서버를 AgentCore 런타임(Runtime)에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore 런타임(Runtime)이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## 원격 액세스를 위한 구성(Configuration) 저장

배포(Deployment)된 MCP 서버를 호출(Invocation)하기 전에 쉽게 검색할 수 있도록 Agent ARN과 Cognito 구성(Configuration)을 AWS Systems Manager Parameter Store와 AWS Secrets Manager에 저장해 보겠습니다:

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## 원격 테스트 클라이언트 생성

이제 배포(Deployment)된 MCP 서버를 테스트할 클라이언트를 생성해 보겠습니다. 이 클라이언트는 AWS에서 필요한 자격 증명을 검색하고 배포(Deployment)된 서버에 연결합니다:

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta
import traceback

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """Refresh access token using refresh token"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """Check token expiry and refresh if needed"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except Exception as e:
        print("🔄 Invalid token, refreshing...", e)
        traceback.print_exc()
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## 배포(Deployment)된 MCP 서버 테스트

원격 클라이언트를 사용하여 배포(Deployment)된 MCP 서버를 테스트해 보겠습니다:

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

## 원격으로 MCP 도구(Tool) 호출(Invocation)

이제 도구(Tool)를 나열할 뿐만 아니라 실제로 호출(Invocation)하여 전체 MCP 기능을 시연하는 향상된 클라이언트를 생성해 보겠습니다:

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """Refresh access token using refresh token"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """Check token expiry and refresh if needed"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except:
        print("🔄 Invalid token, refreshing...")
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## 도구(Tool) 호출(Invocation) 테스트

실제로 호출(Invocation)하여 MCP 도구(Tool)를 테스트해 보겠습니다:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## 다음 단계

AgentCore 런타임(Runtime)에 MCP 서버를 성공적으로 배포(Deployment)했으므로 다음을 수행할 수 있습니다:

1. **더 많은 도구(Tool) 추가**: 추가 도구(Tool)로 MCP 서버 확장
2. **커스텀 인증(Authentication)**: 커스텀 JWT 인가자(Authorizer) 구현
3. **통합**: 다른 AgentCore 서비스와 통합

## 정리 (선택사항)

이 튜토리얼 중에 생성된 리소스를 정리하려면 다음 셀을 실행하세요:

In [ ]:
# print("🗑️  Starting cleanup process...")

# try:
#     ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")

# try:
#     secrets_client.delete_secret(
#         SecretId='mcp_server/cognito/credentials',
#         ForceDeleteWithoutRecovery=True
#     )
#     print("✓ Secrets Manager secret deleted")
# except secrets_client.exceptions.ResourceNotFoundException:
#     print("ℹ️  Secrets Manager secret not found")

# print("\n✅ Cleanup completed successfully!")

In [ ]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

# 축하합니다!

성공적으로 완료한 내용:

✅ **MCP 서버 생성** - 커스텀 도구(Tool) 포함
✅ **로컬에서 테스트** - MCP 클라이언트 사용
✅ **인증(Authentication) 설정** - Amazon Cognito 사용
✅ **AWS에 배포(Deployment)** - AgentCore 런타임(Runtime) 사용
✅ **원격으로 호출(Invocation)** - 적절한 인증(Authentication) 포함
✅ **MCP 개념 학습** - 및 모범 사례

MCP 서버가 이제 Amazon Bedrock AgentCore 런타임(Runtime)에서 실행되고 있으며 프로덕션 사용 준비가 되었습니다!

## 요약

이 튜토리얼에서 배운 내용:
- FastMCP를 사용하여 MCP 서버 구축
- AgentCore 호환성을 위한 무상태 HTTP 전송 구성(Configuration)
- Amazon Cognito를 사용한 JWT 인증(Authentication) 설정
- AWS에서 MCP 서버 배포(Deployment) 및 관리
- 로컬 및 원격 테스트
- 도구(Tool) 호출(Invocation)을 위한 MCP 클라이언트 사용

배포(Deployment)된 MCP 서버는 이제 더 큰 AI 애플리케이션 및 워크플로우에 통합될 수 있습니다!